# Healthcare Triage Goal-Oriented Multi-Agent Flow (CrewAI)

### **Problem Statement**

In hospitals and emergency departments, medical staff must quickly assess patients and determine treatment priority. Manual triage can be inconsistent due to fatigue, workload, or limited medical information. A system is needed that can:

* Analyze symptoms
* Assess severity
* Assign care priority
* Adapt decisions based on patient condition (stable vs. critical)

### **Approach**

This solution uses a **goal-driven multi-agent workflow** where each AI agent has a specialized medical role. The system processes patient input and collaboratively decides urgency using structured reasoning.

The flow operates through the following agents:

1. **Symptom Extraction Agent**
   Identifies key medical symptoms and clinical indicators from free-text input.

2. **Severity Assessment Agent**
   Evaluates symptom risk level and determines potential medical seriousness.

3. **Decision Strategy Agent**
   Uses a hybrid approach:

   * **Reactive Mode:** Immediate action for high-risk cases
   * **Deliberate Mode:** Structured evaluation for non-critical cases

4. **Priority Assignment Agent**
   Assigns a final triage level (Low, Medium, High, Emergency) with justification.

Patient input (`patient_details`) dynamically influences all reasoning steps.

### **Outcome**

The system provides:

✔ Standardized medical triage

✔ Faster response to emergencies

✔ Human-overseeable decision logic

✔ Adaptable planning based on condition severity



For running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [9]:
# ---------------- AGENTS ----------------

triage_agent = Agent(
    role="Triage Specialist",
    goal="Classify patient urgency based on symptoms and reported condition.",
    backstory="You are trained in medical emergency response and follow triage protocols like ESI and CTAS.",
    llm=llm,
    verbose=True
)

planning_agent = Agent(
    role="Care Planning Strategist",
    goal="Generate a structured plan for treatment priorities.",
    backstory="You create evidence-based plans and ensure medical decisions follow best practice frameworks.",
    llm=llm,
    verbose=True
)

decision_agent = Agent(
    role="Hybrid Decision Intelligence Unit",
    goal="Select between reactive emergency response or deliberate planned care.",
    backstory="When a case is critical, you respond rapidly. For stable cases, you perform deeper reasoning.",
    llm=llm,
    verbose=True
)

monitoring_agent = Agent(
    role="Patient Monitoring and Escalation Agent",
    goal="Continuously evaluate patient vitals and symptoms to adjust urgency dynamically.",
    backstory="You track patient condition in real-time and trigger escalation if health indicators deteriorate.",
    llm=llm,
    verbose=True
)



In [18]:
# ---------------- TASKS ----------------

triage_task = Task(
    description=(
        "You will evaluate the provided patient details.\n"
        "Patient Symptoms:\n\n"
        "{patient_details}\n\n"
        "Steps:\n"
        "1. Identify key symptoms.\n"
        "2. Classify severity (Mild / Moderate / Severe / Emergency).\n"
        "3. Recommend next steps.\n\n"
        "Return the result in a clear structured response."
    ),
    expected_output="A structured medical triage report including severity and guidance.",
    agent=triage_agent
)

care_plan_task = Task(
    description="""Using the triage results and patient details "{patient_details}",
generate a treatment prioritization plan with 3–5 actionable steps.
Ensure actions align with evidence-based emergency response standards.""",
    agent=planning_agent,
    expected_output="A structured care plan."
)

decision_task = Task(
    description="""Using the triage results and care plan for "{patient_details}":
Decide the treatment mode:
- 'Reactive Emergency Response' for high or critical cases
- 'Deliberative Planned Care' for stable cases

Provide justification in 2–3 sentences.""",
    agent=decision_agent,
    expected_output="Decision + reasoning."
)

monitoring_task = Task(
    description="""Simulate ongoing condition monitoring for "{patient_details}".
If signs of deterioration appear (e.g., 'breathing difficulty', 'loss of consciousness', 'chest pain worsening'):
Return: "ESCALATE IMMEDIATELY".

Otherwise return: "Patient stable. Continue existing care plan." """,
    agent=monitoring_agent,
    expected_output="Monitoring result and escalation status."
)

In [19]:
# ---------------- CREW PIPELINE ----------------

triage_pipeline = Crew(
    agents=[triage_agent, planning_agent, decision_agent, monitoring_agent],
    tasks=[triage_task, care_plan_task, decision_task, monitoring_task],
    verbose=2
)

2025-12-09 21:15:44,472 - 6464 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
patient_details = """Patient has mild fever, sore throat, and fatigue for two days. No trouble breathing, no chest pain."""

result = triage_pipeline.kickoff(inputs={"patient_details": patient_details})

print("\n--- FINAL OUTPUT ---\n")
print(result)

 [DEBUG]: == Working Agent: Triage Specialist
 [INFO]: == Starting Task: You will evaluate the provided patient details.
Patient Symptoms:

Patient has mild fever, sore throat, and fatigue for two days. No trouble breathing, no chest pain.

Steps:
1. Identify key symptoms.
2. Classify severity (Mild / Moderate / Severe / Emergency).
3. Recommend next steps.

Return the result in a clear structured response.


> Entering new CrewAgentExecutor chain...
I need to evaluate the provided patient details based on their symptoms and classify the urgency of their condition. The patient has mild fever, sore throat, and fatigue for two days, with no trouble breathing or chest pain. 

1. **Identify key symptoms**: 
   - Mild fever
   - Sore throat
   - Fatigue

2. **Classify severity**: 
   - The symptoms presented are mild and do not indicate any severe or emergency conditions. There is no indication of respiratory distress or chest pain, which are critical signs that would elevate the severity c

### Try yourself

Test Case 1 — Low Priority / Stable
Patient has mild fever, sore throat, and fatigue for two days. No trouble breathing, no chest pain.

Expected outcome: Deliberative care plan, stable monitoring.

Test Case 2 — Moderate / Needs Monitoring
Patient reports dizziness, headache, and blurred vision. No vomiting, speaking normally, but off-balance.

Expected outcome: Structured plan + monitoring agent may warn for escalation.

Test Case 3 — High Severity / Potential Emergency
Patient experiencing chest pain radiating to the left arm, shortness of breath, and sweating. Cannot stand without assistance.

Expected outcome: Reactive emergency response + likely escalation.

Test Case 4 — Pediatric Case
A 6-year-old with persistent vomiting, dehydration signs, and lethargy. No trauma or fever, but unable to keep liquids down.

Expected outcome: Moderate–high risk, urgent care plan, continuous monitoring.

Test Case 5 — Trauma
Patient has severe leg injury after a fall. Bleeding controlled but intense pain, swelling, and limited movement. No loss of consciousness.

Expected outcome: Rapid triage + monitoring for shock.